# preamble

In [1]:
########## PREAMBLE ###########################################################

# general imports
import openai
import os
import requests
import json

# import scripts
from scripts import transcribe, process_transcript, check_response, prepare_yield_NOTION
from prompts import prompts

# declaring secrets
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
NOTION_API_KEY = os.environ["NOTION_API_KEY"]
NOTION_DATABASE_ID = os.environ["NOTION_DATABASE_ID"]

# declaring static variables
MAX_WHISPER_AUDIO_SIZE = 26262828

MAX_CONTEXT_4K = 4097
MAX_CONTEXT_16K = 4097 * 4

MAX_BLOCK_SIZE = 2000

# declaring dynamic variables
AUDIO_FILE_PATH = "./to_process/230922-AI_interview-Lisan.mp3"
THOUGHT = "interview_english"

In [2]:
# print all Thought options
for thought in prompts:
    print(thought)

daily_reflection
shower
blog
interview_nederlands
interview_english


# thought processing

### transcription

In [3]:
print("Transcription started.")
transcript = transcribe(THOUGHT, AUDIO_FILE_PATH) # also deals with long audio files
print("Transcription finished!")

Transcription started.
Transcription finished!


In [4]:
transcript

"Oh, there we go. Okay. So, I'm trying to figure out just how people feel about AI. I'm not looking for any specific answers. So, I'm just going to ask you some leading questions. Just to get the conversation started. But feel free to talk about anything. And whatever pops up. So, just to start things off. Oh, friendly. Just to start off. What does AI mean to you? And how do you feel about it? So, AI. I would say... So, it's like an algorithm or software or whatever. Or a system that can learn. Okay. And then when they learn, they can apply their knowledge in a new domain. I think that would be AI to me. Because that's like learning for us as well. So, you learn stuff at school and then you can do it somewhere else. I think that should be artificial intelligence. And how I feel about it. Well, I have to say I don't have really strong feelings about it. Because I think AI... How do you feel about AI? I think it's still really, really broad. If you ask, I don't know, chat GPT. I'm very h

### transcript processing

In [5]:
print("Text processing started.")
text_response = process_transcript(THOUGHT, transcript)
print("Text processing succes!")

Text processing started.
Text processing succes!


In [6]:
print(type(text_response["choices"][0]["message"]["content"]))
text_response["choices"][0]["message"]["content"]

<class 'str'>


'{"title": "Een gesprek over AI, onderwijs en ethiek", "samenvatting": "In dit gesprek wordt AI besproken, evenals de mogelijke impact ervan op jongeren en ouderen. Er wordt onder andere ingegaan op de rol van AI in het onderwijs, de veranderingen die het teweegbrengt in de maatschappij en de ethische aspecten ervan. Bram, de gesprekspartner, vertelt over zijn persoonlijke ervaringen met AI en zijn onderzoek naar ethiek in AI.", "onderwerpen": ["AI", "onderwijs", "ethiek", "verandering", "jongeren", "ouderen", "expertise", "tips"]}'

### check data Notion

In [7]:
content_notion = check_response(text_response)

In [8]:
content_notion

{'title': 'Een gesprek over AI, onderwijs en ethiek',
 'samenvatting': 'In dit gesprek wordt AI besproken, evenals de mogelijke impact ervan op jongeren en ouderen. Er wordt onder andere ingegaan op de rol van AI in het onderwijs, de veranderingen die het teweegbrengt in de maatschappij en de ethische aspecten ervan. Bram, de gesprekspartner, vertelt over zijn persoonlijke ervaringen met AI en zijn onderzoek naar ethiek in AI.',
 'onderwerpen': '- AI\n- onderwijs\n- ethiek\n- verandering\n- jongeren\n- ouderen\n- expertise\n- tips\n'}

### upload to Notion

In [9]:
url = "https://api.notion.com/v1/pages/"

headers =   {
    "Authorization": f"Bearer {NOTION_API_KEY}",
    "Notion-Version": "2022-06-28",
    "Content-Type": "application/json"
            }

data_database = prepare_yield_NOTION(THOUGHT, NOTION_DATABASE_ID, MAX_BLOCK_SIZE,
                                     content_notion, transcript)

response_get = requests.post(url, headers=headers, json=data_database)
print("Processed text uploading succesful!")

Processed text uploading succesful!


In [10]:
response_get.json()

{'object': 'page',
 'id': 'f2869a73-5a32-4d2f-bc07-e9baf71e85ec',
 'created_time': '2023-09-19T06:02:00.000Z',
 'last_edited_time': '2023-09-19T06:02:00.000Z',
 'created_by': {'object': 'user',
  'id': 'dc664599-409b-4fbc-b638-fa34c238bb19'},
 'last_edited_by': {'object': 'user',
  'id': 'dc664599-409b-4fbc-b638-fa34c238bb19'},
 'cover': None,
 'icon': {'type': 'emoji', 'emoji': '🎙️'},
 'parent': {'type': 'database_id',
  'database_id': 'e5568bc0-5015-4ec7-bd8c-ba5050353de6'},
 'archived': False,
 'properties': {'Created time': {'id': 'BAiK',
   'type': 'created_time',
   'created_time': '2023-09-19T06:02:00.000Z'},
  'Thought': {'id': 'FIsG',
   'type': 'select',
   'select': {'id': 'd2e94385-4795-46bb-979a-300e8a1389c4',
    'name': 'nederlands',
    'color': 'orange'}},
  'Name': {'id': 'title',
   'type': 'title',
   'title': [{'type': 'text',
     'text': {'content': 'Een gesprek over AI, onderwijs en ethiek',
      'link': None},
     'annotations': {'bold': False,
      'italic'

---

# Testing

In [13]:
def is_jsonable(x):
    try:
        json.dumps(x)
        return True
    except (TypeError, OverflowError):
        return False

In [40]:
for key in data_database:
    print(is_jsonable(data_database[key]))

True
True
True
True
